### --- Day 17: Clumsy Crucible ---

The lava starts flowing rapidly once the Lava Production Facility is operational. As you leave, the reindeer offers you a parachute, allowing you to quickly reach Gear Island.

As you descend, your bird's-eye view of Gear Island reveals why you had trouble finding anyone on your way up: half of Gear Island is empty, but the half below you is a giant factory city!

You land near the gradually-filling pool of lava at the base of your new lavafall. Lavaducts will eventually carry the lava throughout the city, but to make use of it immediately, Elves are loading it into large crucibles on wheels.

The crucibles are top-heavy and pushed by hand. Unfortunately, the crucibles become very difficult to steer at high speeds, and so it can be hard to go in a straight line for very long.

To get Desert Island the machine parts it needs as soon as possible, you'll need to find the best way to get the crucible from the lava pool to the machine parts factory. To do this, you need to minimize heat loss while choosing a route that doesn't require the crucible to go in a straight line for too long.

Fortunately, the Elves here have a map (your puzzle input) that uses traffic patterns, ambient temperature, and hundreds of other parameters to calculate exactly how much heat loss can be expected for a crucible entering any particular city block.

For example:
```
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
```
Each city block is marked by a single digit that represents the amount of heat loss if the crucible enters that block. The starting point, the lava pool, is the top-left city block; the destination, the machine parts factory, is the bottom-right city block. (Because you already start in the top-left block, you don't incur that block's heat loss unless you leave that block and then return to it.)

Because it is difficult to keep the top-heavy crucible going in a straight line for very long, it can move at most three blocks in a single direction before it must turn 90 degrees left or right. The crucible also can't reverse direction; after entering each city block, it may only turn left, continue straight, or turn right.

One way to minimize heat loss is this path:
```
2>>34^>>>1323
32v>>>35v5623
32552456v>>54
3446585845v52
4546657867v>6
14385987984v4
44578769877v6
36378779796v>
465496798688v
456467998645v
12246868655<v
25465488877v5
43226746555v>
```
This path never moves more than three consecutive blocks in the same direction and incurs a heat loss of only 102.

Directing the crucible from the lava pool to the machine parts factory, but not moving more than three consecutive blocks in the same direction, what is the least heat loss it can incur?


In [1]:
%%time
import queue

f = open("input.txt", "r")

grid = []

total = 0
for line in f:
    line = line.replace('\n', '')
    grid.append(line)
    
# Dijkstra's
visited = set()
distances = {}
prev = {}

directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

for i in range(len(grid)):
    for j in range(len(grid[0])):
        for k in range(4):
            if (j == 0 and k == 0) or (j == len(grid[0]) - 1 and k == 2) \
            or (i == 0 and k == 1) or (i == len(grid) - 1 and k == 3):
                continue
            
            for l in range(1, 4):
                distances[(i,j,k,l)] = 100000

to_visit = queue.PriorityQueue()
                
initial = (0, 0, 0, 0)
distances[initial] = 0
to_visit.put((0, initial))

while not to_visit.empty():
    _, current = to_visit.get()
    #current = min(to_visit, key=lambda n: distances[n] + 282 - n[0] - n[1])
    #to_visit.remove(current)
    visited.add(current)
    
    if current[0] == len(grid) - 1 and current[1] == len(grid[0]) - 1:
        print(current, distances[current])
        break
    
    # Get neighbors
    for d in range(4):
        next_node = None
        dire = directions[d]
        if dire == directions[current[2]]:
            if current[3] == 3:
                continue
            next_node = (current[0] + dire[0], current[1] + dire[1], d, current[3] + 1)
        elif dire[0] == -directions[current[2]][0] and dire[1] == -directions[current[2]][1]:
            continue
        else:
            next_node = (current[0] + dire[0], current[1] + dire[1], d, 1)
            
        if next_node[0] < 0 or next_node[0] >= len(grid) or next_node[1] < 0 or next_node[1] >= len(grid[0]) or next_node == None:
            continue

        # Evaluate tentative distance and swap if less
        if next_node not in visited:
            dist = distances[next_node]
            new_dist = distances[current] + int(grid[next_node[0]][next_node[1]])
            if new_dist < dist:
                distances[next_node] = new_dist
                prev[next_node] = current
                to_visit.put((new_dist + 2*len(grid) - next_node[0] - next_node[1], next_node))

(140, 140, 1, 1) 1110
CPU times: total: 141 ms
Wall time: 1.63 s


### --- Part Two ---

The crucibles of lava simply aren't large enough to provide an adequate supply of lava to the machine parts factory. Instead, the Elves are going to upgrade to ultra crucibles.

Ultra crucibles are even more difficult to steer than normal crucibles. Not only do they have trouble going in a straight line, but they also have trouble turning!

Once an ultra crucible starts moving in a direction, it needs to move a minimum of four blocks in that direction before it can turn (or even before it can stop at the end). However, it will eventually start to get wobbly: an ultra crucible can move a maximum of ten consecutive blocks without turning.

In the above example, an ultra crucible could follow this path to minimize heat loss:
```
2>>>>>>>>1323
32154535v5623
32552456v4254
34465858v5452
45466578v>>>>
143859879845v
445787698776v
363787797965v
465496798688v
456467998645v
122468686556v
254654888773v
432267465553v
```
In the above example, an ultra crucible would incur the minimum possible heat loss of 94.

Here's another example:
```
111111111111
999999999991
999999999991
999999999991
999999999991
```
Sadly, an ultra crucible would need to take an unfortunate path like this one:
```
1>>>>>>>1111
9999999v9991
9999999v9991
9999999v9991
9999999v>>>>
```
This route causes the ultra crucible to incur the minimum possible heat loss of 71.

Directing the ultra crucible from the lava pool to the machine parts factory, what is the least heat loss it can incur?


In [7]:
%%time
# Ultra Crucible
import queue

file = "input.txt"
goal = (140, 140)

test = False
if test:
    file = "test.txt"
    goal = (12, 12)

mov_min = 4
mov_max = 10
    
size = goal[0] + 1
grid = {(i,j): int(c) for i,r in enumerate(open(file)) for j,c in enumerate(r.strip())}
    
# Dijkstra's
visited = set()
distances = {}
prev = {}

directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

for key in grid:
        for k in range(4):          
            for l in range(mov_min, mov_max + 1):
                distances[key[0], key[1], k, l] = 100000

to_visit = queue.PriorityQueue()
                
initial = (0, 0, 0, 0)
distances[initial] = 0
to_visit.put((0, initial))

while not to_visit.empty():
    _, current = to_visit.get()
    visited.add(current)
    
    if (current[0], current[1]) == goal:
        print(current, distances[current])
        break
    
    # Get neighbors
    for d in range(4):
        if d == (current[2] + 2) % 4: # Backwards
            continue
            
        next_node = None
        dist = 0
        dire = directions[d]
        
        if dire == directions[current[2]]: # Forward
            if current[3] == 0: # For the initial node
                next_node = (current[0] + mov_min*dire[0], current[1] + mov_min*dire[1], d, mov_min)
                if (next_node[0], next_node[1]) not in grid:
                    continue    
                for i in range(mov_min):
                    dist += grid[current[0] + (i+1)*dire[0], current[1] + (i+1)*dire[1]]
            elif current[3] == mov_max:
                continue
            else:
                next_node = (current[0] + dire[0], current[1] + dire[1], d, current[3] + 1)
                if (next_node[0], next_node[1]) not in grid:
                    continue
                dist = grid[next_node[0], next_node[1]]
        else: # Turn L or R
            next_node = (current[0] + mov_min*dire[0], current[1] + mov_min*dire[1], d, mov_min)
            if (next_node[0], next_node[1]) not in grid:
                    continue    
            for i in range(mov_min):
                dist += grid[current[0] + (i+1)*dire[0], current[1] + (i+1)*dire[1]]

        # Evaluate tentative distance and swap if less
        if next_node not in visited:
            distance = distances[next_node]
            new_dist = distances[current] + dist
            if new_dist < distance:
                distances[next_node] = new_dist
                prev[next_node] = current
                to_visit.put((new_dist + 2*size - next_node[0] - next_node[1], next_node))

(140, 140, 1, 9) 1294
CPU times: total: 906 ms
Wall time: 4.78 s


In [371]:
dist = (140, 140, 1, 9)
while dist != (0, 0, 0, 0):
    print(dist, distances[dist])
    dist = prev[dist]
print(dist, distances[dist])

(140, 140, 1, 9) 1294
(139, 140, 1, 8) 1290
(138, 140, 1, 7) 1286
(137, 140, 1, 6) 1282
(136, 140, 1, 5) 1278
(135, 140, 1, 4) 1273
(131, 140, 0, 6) 1263
(131, 139, 0, 5) 1261
(131, 138, 0, 4) 1260
(131, 134, 1, 10) 1248
(130, 134, 1, 9) 1245
(129, 134, 1, 8) 1244
(128, 134, 1, 7) 1238
(127, 134, 1, 6) 1231
(126, 134, 1, 5) 1229
(125, 134, 1, 4) 1228
(121, 134, 2, 5) 1216
(121, 135, 2, 4) 1215
(121, 139, 1, 9) 1203
(120, 139, 1, 8) 1196
(119, 139, 1, 7) 1189
(118, 139, 1, 6) 1187
(117, 139, 1, 5) 1181
(116, 139, 1, 4) 1176
(112, 139, 0, 4) 1168
(112, 135, 1, 9) 1155
(111, 135, 1, 8) 1150
(110, 135, 1, 7) 1148
(109, 135, 1, 6) 1144
(108, 135, 1, 5) 1139
(107, 135, 1, 4) 1134
(103, 135, 2, 4) 1119
(103, 139, 1, 9) 1105
(102, 139, 1, 8) 1103
(101, 139, 1, 7) 1099
(100, 139, 1, 6) 1098
(99, 139, 1, 5) 1091
(98, 139, 1, 4) 1090
(94, 139, 0, 5) 1080
(94, 138, 0, 4) 1073
(94, 134, 1, 9) 1055
(93, 134, 1, 8) 1053
(92, 134, 1, 7) 1046
(91, 134, 1, 6) 1044
(90, 134, 1, 5) 1040
(89, 134, 1, 4) 10

In [369]:
dist = (140, 140, 1, 9)
while dist != (0, 0, 3, 0):
    print(dist, distances[dist])
    dist = prev[dist]
print(dist, distances[dist])

(140, 140, 1, 9) 1304
(139, 140, 1, 8) 1300
(138, 140, 1, 7) 1296
(137, 140, 1, 6) 1292
(136, 140, 1, 5) 1288
(135, 140, 1, 4) 1283
(131, 140, 0, 6) 1273
(131, 139, 0, 5) 1271
(131, 138, 0, 4) 1270
(131, 134, 1, 10) 1258
(130, 134, 1, 9) 1255
(129, 134, 1, 8) 1254
(128, 134, 1, 7) 1248
(127, 134, 1, 6) 1241
(126, 134, 1, 5) 1239
(125, 134, 1, 4) 1238
(121, 134, 2, 5) 1226
(121, 135, 2, 4) 1225
(121, 139, 1, 9) 1213
(120, 139, 1, 8) 1206
(119, 139, 1, 7) 1199
(118, 139, 1, 6) 1197
(117, 139, 1, 5) 1191
(116, 139, 1, 4) 1186
(112, 139, 0, 4) 1178
(112, 135, 1, 9) 1165
(111, 135, 1, 8) 1160
(110, 135, 1, 7) 1158
(109, 135, 1, 6) 1154
(108, 135, 1, 5) 1149
(107, 135, 1, 4) 1144
(103, 135, 2, 4) 1129
(103, 139, 1, 9) 1115
(102, 139, 1, 8) 1113
(101, 139, 1, 7) 1109
(100, 139, 1, 6) 1108
(99, 139, 1, 5) 1101
(98, 139, 1, 4) 1100
(94, 139, 0, 5) 1090
(94, 138, 0, 4) 1083
(94, 134, 1, 9) 1065
(93, 134, 1, 8) 1063
(92, 134, 1, 7) 1056
(91, 134, 1, 6) 1054
(90, 134, 1, 5) 1050
(89, 134, 1, 4) 10

In [366]:
%%time
# Ultra Crucible Cleaner Attempt (Not working yet)
import queue

test = False

file = "input.txt"
goal = (140, 140)

if test:
    file = "test.txt"
    goal = (12, 12)
    

grid = {(i,j): int(c) for i,r in enumerate(open(file)) for j,c in enumerate(r.strip())}

mov_min = 4
mov_max = 10

# A* algorithm
visited = set()
to_visit = queue.PriorityQueue()

# Bookkeeping
distances = {}
prev = {}

directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

for key in grid:
    distances[key] = 100000
                
initial = (0, 0, 3, 0) # pos0, pos1, dir, moved
distances[(0,0)] = 0
to_visit.put((0, initial))

while not to_visit.empty():
    cur_dist, current = to_visit.get()
    
    if current in visited:
        continue
        
    visited.add(current)
    
    y, x, cur_dir, moved = current
    
    
    if goal == (y, x):
        print(current, cur_dist)
        break
    
    # Get neighbors
    for d in range(4):
        if d == (cur_dir + 2) % 4 or d == cur_dir: # Backwards or Forwards
            continue
        else: # Turn L or R
            dire = directions[d]
            dist = 0
            for i in range(1, mov_max + 1):
                next_pos = (current[0] + i*dire[0], current[1] + i*dire[1])
                if next_pos not in grid:
                    break # CONFIRM
                dist += grid[next_pos]
                print(i, dist)
                if i >= mov_min:
                    next_node = (next_pos[0], next_pos[1], d, i)
                    to_visit.put((cur_dist + dist, next_node))
                    print(next_node, cur_dist + dist, dist)
                    RETURN

1 5
2 9
3 15
4 20
(0, 4, 0, 4) 20 20


NameError: name 'RETURN' is not defined

In [346]:
for i in range(mov_min, mov_max + 1):
    print(i)

4
5
6
7
8
9
10


In [293]:
dist = prev[(140, 140, 1, 9)]
print(dist, distances[dist[0], dist[1]])

while dist != (0, 0, 3, 0):
    dist = prev[dist]
    print(dist, distances[dist[0], dist[1]])

(131, 140, 0, 6) 1253
(131, 134, 1, 9) 1238
(122, 134, 2, 6) 1211
(122, 140, 1, 7) 1203
(115, 140, 0, 4) 1175
(115, 136, 1, 7) 1166
(108, 136, 2, 4) 1145
(108, 140, 1, 10) 1134
(98, 140, 0, 5) 1102
(98, 135, 1, 7) 1086
(91, 135, 0, 6) 1054
(91, 129, 1, 10) 1026
(81, 129, 2, 6) 986
(81, 135, 1, 7) 977
(74, 135, 0, 4) 951
(74, 131, 1, 10) 935
(64, 131, 2, 4) 894
(64, 135, 1, 9) 881
(55, 135, 0, 5) 844
(55, 130, 1, 10) 818
(45, 130, 0, 5) 774
(45, 125, 1, 8) 755
(37, 125, 0, 6) 725
(37, 119, 1, 7) 704
(30, 119, 0, 7) 677
(30, 112, 1, 7) 651
(23, 112, 0, 9) 625
(23, 103, 1, 5) 589
(18, 103, 0, 8) 566
(18, 95, 1, 5) 528
(13, 95, 0, 8) 507
(13, 87, 1, 4) 477
(9, 87, 0, 7) 462
(9, 80, 3, 4) 428
(13, 80, 0, 10) 413
(13, 70, 1, 4) 367
(9, 70, 0, 4) 353
(9, 66, 1, 4) 341
(5, 66, 0, 10) 320
(5, 56, 1, 4) 286
(1, 56, 0, 8) 274
(1, 48, 3, 6) 237
(7, 48, 0, 7) 231
(7, 41, 1, 4) 199
(3, 41, 0, 9) 186
(3, 32, 3, 4) 153
(7, 32, 0, 8) 162
(7, 24, 1, 6) 131
(1, 24, 0, 10) 109
(1, 14, 3, 4) 72
(5, 14, 0, 